In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [3]:
raw_text = open('fungene_9.5_ACT_575_aligned_protein_seqs.fa').read()
raw_text = raw_text.lower()

In [4]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [5]:
chars

['\n',
 ' ',
 '#',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '=',
 '>',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [6]:
len(chars)

49

In [7]:
len(raw_text)

309468

In [8]:
seq_length = 100
x = []
y = []
for i in range(0, len(raw_text) - seq_length):
    given = raw_text[i:i + seq_length]
    predict = raw_text[i + seq_length]
    x.append([char_to_int[char] for char in given])
    y.append(char_to_int[predict])

In [9]:
print(x[:3])
print(y[:3])

[[21, 33, 48, 40, 14, 14, 10, 15, 12, 1, 1, 25, 37, 26, 27, 26, 22, 24, 47, 20, 13, 18, 19, 10, 18, 8, 8, 14, 10, 10, 15, 13, 6, 37, 40, 29, 23, 36, 31, 41, 35, 20, 27, 36, 42, 27, 40, 37, 24, 23, 25, 42, 27, 40, 1, 23, 41, 24, 43, 40, 31, 23, 27, 6, 26, 27, 28, 31, 36, 31, 42, 31, 37, 36, 20, 25, 34, 23, 41, 41, 1, 25, 1, 24, 27, 42, 23, 7, 34, 23, 25, 42, 23, 35, 23, 41, 27, 0, 8, 8], [33, 48, 40, 14, 14, 10, 15, 12, 1, 1, 25, 37, 26, 27, 26, 22, 24, 47, 20, 13, 18, 19, 10, 18, 8, 8, 14, 10, 10, 15, 13, 6, 37, 40, 29, 23, 36, 31, 41, 35, 20, 27, 36, 42, 27, 40, 37, 24, 23, 25, 42, 27, 40, 1, 23, 41, 24, 43, 40, 31, 23, 27, 6, 26, 27, 28, 31, 36, 31, 42, 31, 37, 36, 20, 25, 34, 23, 41, 41, 1, 25, 1, 24, 27, 42, 23, 7, 34, 23, 25, 42, 23, 35, 23, 41, 27, 0, 8, 8, 8], [48, 40, 14, 14, 10, 15, 12, 1, 1, 25, 37, 26, 27, 26, 22, 24, 47, 20, 13, 18, 19, 10, 18, 8, 8, 14, 10, 10, 15, 13, 6, 37, 40, 29, 23, 36, 31, 41, 35, 20, 27, 36, 42, 27, 40, 37, 24, 23, 25, 42, 27, 40, 1, 23, 41, 24, 43,

In [10]:
n_patterns = len(x)
n_vocab = len(chars)

# 把x变成LSTM需要的样子
x = numpy.reshape(x, (n_patterns, seq_length, 1))
# 简单normal到0-1之间
x = x / float(n_vocab)
# output变成one-hot
y = np_utils.to_categorical(y)

print(x[11])
print(y[11])

[[ 0.51020408]
 [ 0.75510204]
 [ 0.53061224]
 [ 0.55102041]
 [ 0.53061224]
 [ 0.44897959]
 [ 0.48979592]
 [ 0.95918367]
 [ 0.40816327]
 [ 0.26530612]
 [ 0.36734694]
 [ 0.3877551 ]
 [ 0.20408163]
 [ 0.36734694]
 [ 0.16326531]
 [ 0.16326531]
 [ 0.28571429]
 [ 0.20408163]
 [ 0.20408163]
 [ 0.30612245]
 [ 0.26530612]
 [ 0.12244898]
 [ 0.75510204]
 [ 0.81632653]
 [ 0.59183673]
 [ 0.46938776]
 [ 0.73469388]
 [ 0.63265306]
 [ 0.83673469]
 [ 0.71428571]
 [ 0.40816327]
 [ 0.55102041]
 [ 0.73469388]
 [ 0.85714286]
 [ 0.55102041]
 [ 0.81632653]
 [ 0.75510204]
 [ 0.48979592]
 [ 0.46938776]
 [ 0.51020408]
 [ 0.85714286]
 [ 0.55102041]
 [ 0.81632653]
 [ 0.02040816]
 [ 0.46938776]
 [ 0.83673469]
 [ 0.48979592]
 [ 0.87755102]
 [ 0.81632653]
 [ 0.63265306]
 [ 0.46938776]
 [ 0.55102041]
 [ 0.12244898]
 [ 0.53061224]
 [ 0.55102041]
 [ 0.57142857]
 [ 0.63265306]
 [ 0.73469388]
 [ 0.63265306]
 [ 0.85714286]
 [ 0.63265306]
 [ 0.75510204]
 [ 0.73469388]
 [ 0.40816327]
 [ 0.51020408]
 [ 0.69387755]
 [ 0.46938

In [11]:
model = Sequential()
model.add(LSTM(128, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [12]:
model.fit(x, y, nb_epoch=10, batch_size=32)

D:\Anaconda\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
309368/309368 [==============================] - 1703s 6ms/step - loss: 2.5672
Epoch 2/10
309368/309368 [==============================] - 1703s 6ms/step - loss: 1.6143
Epoch 3/10
309368/309368 [==============================] - 1703s 6ms/step - loss: 1.1188
Epoch 4/10
309368/309368 [==============================] - 1660s 5ms/step - loss: 2.0657
Epoch 5/10
309368/309368 [==============================] - 1651s 5ms/step - loss: 1.4720
Epoch 6/10
309368/309368 [==============================] - 1711s 6ms/step - loss: 1.9694
Epoch 7/10
309368/309368 [==============================] - 1721s 6ms/step - loss: 1.8574
Epoch 8/10
309368/309368 [==============================] - 1777s 6ms/step - loss: 2.4625
Epoch 9/10
309368/309368 [==============================] - 1764s 6ms/step - loss: 2.2191
Epoch 10/10
309368/309368 [==============================] - 1996s 6ms/step - loss: 1.3554
